In [144]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import yfinance as yf
from yahoo_fin import stock_info
from Modules.preprocess import process_data
from copy import deepcopy as dc

In [145]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

## Class For a Preprocessed Dataset

In [146]:
class PPDataset:
    def __init__(self, x, y, x_train, x_test, y_train, y_test, scaler):
        self._x = x
        self._y = y
        self.x_train = x_train
        self.x_test = x_test
        self.y_train = y_train
        self.y_test = y_test
        self.scaler = scaler
    
    def get_x_train(self):
        return self.x_train
    
    def get_x_test(self):
        return self.x_test
    
    def get_y_train(self):
        return self.y_train
    
    def get_y_test(self):
        return self.y_test
    
    def get_scaler(self):
        return self.scaler
    
    

## Function To Loop the Preprocessing Of .csv Files

In [147]:
def prepare_data_for_metrics(tickers):
    # Create list to store preprocessed datasets
    preparedDataSets = list()
    # Loop preprocess all datasets
    for ticker in tickers:
        file = '../../current/'+ticker+'.csv'
        historical_stock_prices = (yf.Ticker(ticker)).history(period = 'max')
        historical_stock_prices.to_csv(file)
        X, y, X_train, X_test, y_train, y_test, scaler = process_data(file)
        ppd = PPDataset(X, y, X_train, X_test, y_train, y_test, scaler)
        preparedDataSets.append(ppd)
    # return list
    return preparedDataSets

## Function To Loop the Processing of Test Predictions

In [148]:
def get_test_predictions(preparedDataSets, lookback, outmodel):
    test_preds_list = list()
    for dataset in preparedDataSets:
        x_test = dataset.get_x_test()
        scaler = dataset.get_scaler()
        test_predictions = outmodel(x_test.to(device)).detach().cpu().numpy().flatten()

        dummies = np.zeros((x_test.shape[0], lookback+1))
        dummies[:, 0] = test_predictions
        dummies = scaler.inverse_transform(dummies)

        test_predictions = dc(dummies[:, 0])
        test_preds_list.append(test_predictions)

    return test_preds_list

## Functions To Output RMSE

In [149]:
# Python program to get average of a list 
def average(lst):
    return sum(lst) / len(lst)

def output_rmse(tickers, test_predictions, preparedDataSets):
    i = 0
    rmse_list = list()
    while i < len(tickers): 
        # Convert test prediction to np array
        test_prediction_np = np.array(test_predictions[i])
        # Call prepared datasets[i]
        preppedDataset = preparedDataSets[i]
        # Convert y_test to np array
        y_test_np = np.array(preppedDataset.get_y_test())
        # Calculate RMSE
        rmse = np.sqrt(np.mean((test_prediction_np - y_test_np) ** 2))
        # print rmse for ticker
        print("RMSE for "+str(tickers[i])+": "+str(rmse))
        rmse_list.append(rmse)
        i = i + 1
    avg_rmse = average(rmse_list)
    print("\nAverage RMSE:", avg_rmse)

## Gather Preprocessed Data Into List

In [150]:
tickers = ['AAPL','AMZN','BRK','META','SNAP','BK']


preparedDataSets = prepare_data_for_metrics(tickers)

# preparedDataSets

/Users/varad/Documents/2023fall/Deep Learning/final_project/github/Stock_Forecasting_DL/Modules/preprocess.py:23: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  data['Date'] = pd.to_datetime(data['Date'])
/Users/varad/Documents/2023fall/Deep Learning/final_project/github/Stock_Forecasting_DL/Modules/preprocess.py:23: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  data['Date'] = pd.to_datetime(data['Date'])
/Users/varad/Documents/2023fall/Deep L

## Load Model From pt File

In [151]:
outmodel = torch.load('forecast_lstm_10.pt',  map_location=torch.device('cpu'))
outmodel.eval()

LSTM(
  (conv1d): Conv1d(30, 1, kernel_size=(1,), stride=(2,))
  (init_linear): Linear(in_features=1, out_features=1, bias=True)
  (lstm): LSTM(1, 64, num_layers=2, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=128, out_features=1, bias=True)
  (sequential): Sequential(
    (0): Linear(in_features=1, out_features=1, bias=True)
    (1): ReLU()
    (2): LSTM(1, 64, num_layers=2, batch_first=True, bidirectional=True)
  )
)

## Get Test Predictions

In [152]:
lookback = 30

test_predictions = get_test_predictions(preparedDataSets ,lookback, outmodel)
    

## Output RMSE

In [153]:
output_rmse(tickers, test_predictions, preparedDataSets)



RMSE for AAPL: 161.92311547834356
RMSE for AMZ: 923.1076060655041
RMSE for BRK: 0.06119366208150135
RMSE for META: 300.6938114554569
RMSE for SNAP: 10.139855209359665
RMSE for BK: 45.41654707844022

Average RMSE: 240.22368815819766
